In [1]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
#from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import nltk

In [2]:
df = pd.read_csv("Tweets_part.csv")

In [3]:
new_df = df[['words','label','text']]

In [4]:
import nltk
from nltk.tokenize import WordPunctTokenizer

tokenizer = nltk.RegexpTokenizer(r"\w+")



new_df['text'] = new_df['text'].apply(lambda x: tokenizer.tokenize(x.lower()))

C:\Users\kenny\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
from nltk.corpus import stopwords
stopWords = stopwords.words('english')
new_df['text'] = new_df['text'].apply(lambda x: [item for item in x if item not in stopWords])

C:\Users\kenny\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
new_df['words'] = 1

C:\Users\kenny\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
new_df

,words,label,text
0,1,Negative,"[tiffanylue, know, listenin, bad, habit, earli..."
1,1,Neutral,"[layin, n, bed, headache, ughhhh, waitin, call]"
2,1,Negative,"[funeral, ceremony, gloomy, friday]"
3,1,Positive,"[wants, hang, friends, soon]"
4,1,Positive,"[dannycastillo, want, trade, someone, houston,..."
...,...,...,...
39995,1,Neutral,[johnlloydtaylor]
39996,1,Positive,"[happy, mothers, day, love]"
39997,1,Positive,"[happy, mother, day, mommies, woman, man, long..."
39998,1,Positive,"[niariley, wassup, beautiful, follow, peep, ne..."


In [8]:
#à tester
# Lemmatize with POS Tag
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

for index,entry in enumerate(new_df['text']):
    words = []
    
    for word in entry:
        
        pos_word = get_wordnet_pos(word)
        words.append(lemmatizer.lemmatize(word,pos_word))
    new_df.loc[index,'words'] = str(words)
   

C:\Users\kenny\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\kenny\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\kenny\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [9]:
new_df

,words,label,text
0,"['tiffanylue', 'know', 'listenin', 'bad', 'hab...",Negative,"[tiffanylue, know, listenin, bad, habit, earli..."
1,"['layin', 'n', 'bed', 'headache', 'ughhhh', 'w...",Neutral,"[layin, n, bed, headache, ughhhh, waitin, call]"
2,"['funeral', 'ceremony', 'gloomy', 'friday']",Negative,"[funeral, ceremony, gloomy, friday]"
3,"['want', 'hang', 'friend', 'soon']",Positive,"[wants, hang, friends, soon]"
4,"['dannycastillo', 'want', 'trade', 'someone', ...",Positive,"[dannycastillo, want, trade, someone, houston,..."
...,...,...,...
39995,['johnlloydtaylor'],Neutral,[johnlloydtaylor]
39996,"['happy', 'mother', 'day', 'love']",Positive,"[happy, mothers, day, love]"
39997,"['happy', 'mother', 'day', 'mommy', 'woman', '...",Positive,"[happy, mother, day, mommies, woman, man, long..."
39998,"['niariley', 'wassup', 'beautiful', 'follow', ...",Positive,"[niariley, wassup, beautiful, follow, peep, ne..."


In [10]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(new_df['words'],new_df['label'],test_size=0.2)

In [11]:
encoder = LabelEncoder()
train_y_encoded = encoder.fit_transform(train_y)
test_y_encoded = encoder.fit_transform(test_y)

In [12]:
tfidf_vect = TfidfVectorizer(max_features=2000)
tfidf_vect.fit(new_df['words'])
train_x_tfidf = tfidf_vect.transform(train_x)
test_x_tfidf = tfidf_vect.transform(test_x)

In [13]:

naive = naive_bayes.MultinomialNB(alpha=0.5)
naive.fit(train_x_tfidf,train_y_encoded)

predictions_NB = naive.predict(test_x_tfidf)

print("Naive Bayes Accuracy Score: ", accuracy_score(predictions_NB, test_y_encoded)*100)

Naive Bayes Accuracy Score:  74.925


In [14]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(train_x_tfidf,train_y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(test_x_tfidf)
# Use accuracy_score function to get the accuracy
accuracy = accuracy_score(predictions_SVM, test_y)
#print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, test_y)*100)
#print("SVM Accuracy Score -> ",accuracy*100)

print("SVM Accuracy Score -> ",accuracy*100)

SVM Accuracy Score ->  91.7875


In [15]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(random_state=25, solver='lbfgs',max_iter=1000,multi_class='multinomial')
lr.fit(train_x_tfidf,train_y)
pred_train_base=lr.predict(train_x_tfidf)
pred_test_base=lr.predict(test_x_tfidf)

In [16]:
accuracy = accuracy_score(test_y,pred_test_base)

In [17]:
print("LR Accuracy Score -> ",accuracy*100)

LR Accuracy Score ->  90.2625


In [18]:
from sklearn.metrics import classification_report

In [19]:
print(classification_report(test_y, lr.predict(test_x_tfidf)))

              precision    recall  f1-score   support

    Negative       0.90      0.78      0.84      1783
     Neutral       0.84      0.95      0.89      2175
    Positive       0.94      0.93      0.94      4042

    accuracy                           0.90      8000
   macro avg       0.89      0.89      0.89      8000
weighted avg       0.90      0.90      0.90      8000



In [20]:
print(classification_report(test_y, SVM.predict(test_x_tfidf)))

              precision    recall  f1-score   support

    Negative       0.90      0.83      0.86      1783
     Neutral       0.88      0.95      0.92      2175
    Positive       0.95      0.94      0.94      4042

    accuracy                           0.92      8000
   macro avg       0.91      0.91      0.91      8000
weighted avg       0.92      0.92      0.92      8000

